**Chatbot con Generación Aumentada de Recuperación**

En esta sección, demostramos algunos casos de uso comunes de incrustación con algunos ejemplos simples. La distancia euclidiana se utiliza para calcular la similitud entre dos fragmentos de texto.

Búsqueda y recomendación
Supongamos que tiene una colección de documentos (el conjunto de datos). Cada documento está representado por su incrustación. Se le ha proporcionado una cadena de consulta. La solicitud es identificar el documento que es más relevante para la cadena de consulta. Puede lograr esto con los siguientes pasos:

In [3]:
import json
import boto3

bedrock = boto3.client(
    service_name='bedrock-runtime'
)
modelId = 'ai21.j2-ultra'
accept = 'application/json'
contentType = 'application/json'
prompt = """¿Quien era Albert Eistein?"""

input = {'prompt': prompt, 'maxTokens': 200}
body=json.dumps(input)
response = bedrock.invoke_model(body=body, modelId=modelId, accept=accept,contentType=contentType)
response_body = json.loads(response.get('body').read())
completions = response_body['completions']
for part in completions:
    print(part['data']['text'])


Albert Einstein fue un físico y matemático que nació el 14 de marzo de 1879 en Ulm (Alemania). Es conocido sobre todo por sus contribuciones a la mecánica cuántica, la física relativista y sobre todo por el Teorema de Einstein de Relatividad.


Suponiendo que conozcas a otro Ablert Einstein. 

Este Einstein tiene el mismo nombre que el mundialmente famoso Einstein, pero vende conchas marinas en todo el mundo. Piense en el hecho de que los modelos de fundación se entrenan con información o conocimiento de dominio público. Este Einstein, aunque su negocio de conchas marinas puede ser bastante exitoso, pero hay muy poca información o conocimiento sobre él y su negocio en el dominio público. Por esta razón, es poco probable que el modelo de fundación sea consciente de este Einstein y genere respuestas relacionadas con este Einstein, en comparación con ese Einstein.

Sin embargo, si proporcionamos alguna información o conocimiento adicional sobre este Einstein en el prompt, el modelo básico entonces sabe que estamos preguntando sobre este Einstein, no sobre ese Einstein. La información o el conocimiento adicional en el mensaje generalmente se conoce como el contexto de la conversación.

A continuación se muestra un mensaje con contexto adicional:

In [6]:
prompt = """Utiliza el siguiente contexto para responder a la pregunta:

=== Contexto ===
Albert Einstein vende conchas marinas en Los Ángeles.
Albert Einstein regenta una tienda en Los Ángeles.
Albert Einstein vende conchas marinas en Sydney.
Albert Einstein vende conchas marinas en Seúl.
Albert Einstein vende conchas marinas en Pekín.

=== Pregunta ===
¿A qué se dedica Albert Einstein?
            """

input = {'prompt': prompt, 'maxTokens': 200}
body=json.dumps(input)
response = bedrock.invoke_model(body=body, modelId=modelId, accept=accept,contentType=contentType)
response_body = json.loads(response.get('body').read())
completions = response_body['completions']
for part in completions:
    print(part['data']['text'])


 
No rinde ninguna de las respuestas correctas
Albert Einstein se dedico a la física y el estudio de la relatividad general.


Ahora te preguntarás, si el usuario que hace la pregunta ya sabe que Albert Einstein vende conchas marinas en todo el mundo, ¿por qué le preguntaría a un chatbot? En realidad, hacemos una pregunta cuando no sabemos la respuesta, no cuando sabemos la respuesta.

Suponiendo que tengamos alguna información o conocimiento sobre este Albert Einstein y los pongamos a disposición del chatbot. El usuario no tiene acceso a dicha información o conocimiento, pero puede preguntar al chatbot. ¿Cómo puede el chatbot aprovechar la información/conocimiento adicional para proporcionar respuestas sensibles al contexto al usuario?

**Generación aumentada de recuperación (RAG)**

Supongamos que tenemos una base de conocimientos con alguna información o conocimiento que no está disponible en el dominio público. Con la generación aumentada de recuperación, utilizamos los siguientes pasos para generar respuestas sensibles al contexto:

Convierta el mensaje (texto de la pregunta) en incrustación.

- (R) Recupere N entradas más relevantes de la base de conocimientos. Esto se trata como el contexto de la conversación.
- (A) Aumente la indicación con el contexto anteponiendo el contexto al texto de la pregunta. Este resultado final es el mensaje contextual.
- (G) Genere la respuesta alimentando el mensaje sensible al contexto al modelo básico.

En este ejercicio, usamos la colección OpenSearch Serverless como base de conocimiento. Tenga en cuenta que en el índice de demostración ya tenemos algunos datos falsos de **dataset.json**

En el código de ejemplo siguiente se muestra cómo realizar la generación aumentada de recuperación. El texto de la consulta es "¿Qué hace Albert Einstein?" y queremos las 5 entradas más relevantes de la base de conocimiento como contexto.

In [24]:
import boto3
import json
import requests
from requests_aws4auth import AWS4Auth

region = 'us-west-2'
host = 'https://5tl3o6l27r1pcm2hddk9.us-west-2.aoss.amazonaws.com'

def get_embedding(bedrock, text):
    modelId = 'amazon.titan-embed-text-v1'
    accept = 'application/json'
    contentType = 'application/json'
    input = {
            'inputText': text
        }
    body=json.dumps(input)
    response = bedrock.invoke_model(
        body=body, modelId=modelId, accept=accept,contentType=contentType)
    response_body = json.loads(response.get('body').read())
    embedding = response_body['embedding']
    return embedding
    
def search(embedding, limit=1):
    # prepare for OpenSearch Serverless
    service = 'aoss'
    credentials = boto3.Session().get_credentials()
    awsauth = AWS4Auth(
        credentials.access_key, 
        credentials.secret_key, 
        "us-west-2", 
        service, 
        session_token=credentials.token
    )
    # search
    index = 'demo-index'
    datatype = '_search'
    url = host + '/' + index + '/' + datatype
    headers = {'Content-Type': 'application/json'}
    document = {
        'size': limit,
        'query': {
            'knn': {
                'embedding': {
                    'vector': embedding,
                    'k': limit
                }
            }
        }
    }
    # response
    response = requests.get(url, auth=awsauth, json=document, headers=headers)
    response.raise_for_status()
    data = response.json()
    output = ''
    for item in data['hits']['hits']:
        output += item['_source']['content'] + '\n'
    return output

# main function
bedrock = boto3.client(
    service_name='bedrock-runtime'
)
# this is the original prompt (query text)
prompt = 'What does Albert Einstein do?'
# convet the query text into embedding
embedding = get_embedding(bedrock, prompt)
# retrieve 5 most relevant entries from the knowledge base
info = search(embedding, limit=5)
# augment the prompt with the context
prompt = 'Use the context below to answer the question:\n\n=== Context ===\n{0}\n\n=== Question ===\n{1}'.format(info, prompt)
# ask the foundation model
modelId = 'ai21.j2-ultra'
accept = 'application/json'
contentType = 'application/json'
input = {'prompt': prompt, 'maxTokens': 200}
body=json.dumps(input)
response = bedrock.invoke_model(body=body, modelId=modelId, accept=accept,contentType=contentType)
response_body = json.loads(response.get('body').read())
completions = response_body['completions']
for part in completions:
    print(part['data']['text'])



He is a support engineer


**Integración de chatbot**

Ahora modificaremos el código de nuestra función Lambda para agregar la capacidad de realizar la generación aumentada de recuperación. Si el mensaje humano comienza con, se trata como una solicitud de generación aumentada de recuperación. El chatbot extrae el contenido después del comando como texto de consulta, busca N entradas más relevantes de la base de conocimiento, utiliza el conocimiento como contexto para interactuar con el modelo base, devuelve la respuesta del modelo base al usuario final.//rag

**Código de función de Lambda**

Navegue a la carpeta **Chatbot**, use el menú **Archivo -> Nuevo -> Archivo de Python** para crear un nuevo archivo de Python. Esto crea un archivo de Python sin título en la carpeta **Chatbot**. Cambie el nombre del archivo a **chatbot_v3.py**. Copie y pegue el siguiente contenido en **chatbot_v3.py**. Utilice el menú **Archivo -> Guardar archivo de Python** para guardar el contenido del archivo.

In [ ]:
import json
import boto3
import os
import requests
from requests_aws4auth import AWS4Auth

region = os.environ.get('AWS_REGION')
bedrock = boto3.client(service_name='bedrock-runtime')
aoss_host = os.environ.get('aossHost')

def lambda_handler(event, context):
    if (event['httpMethod'] == 'GET'):
        output = load_html()
        return {
            'statusCode': 200,
            'headers': {'Content-Type': 'text/html'},
            'body': output
        }
    elif (event['httpMethod'] == 'POST'):
        prompt = event['body']
        comm, query = check_prompt_command(prompt)
        if comm == 'search':
            output = search(query, limit=5)
        elif comm == 'rag':
            # retrieve the context
            info = search(query, limit=5)
            # augment the prompt with the context
            prompt = 'Use the context below to answer the question:\n\n=== Context ===\n{0}\n\n=== Question ===\n{1}'.format(info, query)
            output = chat(prompt)
        else:
            output = chat(prompt)
        return {
            'statusCode': 200,
            'headers': {'Content-Type': 'text/html'},
            'body': output
        }
    else:
         return {
            'statusCode': 200,
            'headers': {'Content-Type': 'text/html'},
            'body': "OK"
        }

def check_prompt_command(prompt):
    comm = 'chat'
    query = None
    # Check the last line of the prompt to see if it is a search request.
    lines = prompt.splitlines()
    last_line = lines[-1]
    # Check if the last line starts with "Human: "
    if last_line.startswith('Human: '):
        last_line = last_line[7:].strip()
        # Check if the human prompt starts with "//search "
        if last_line.startswith('//search '):
            query = last_line[9:].strip()
            if query != None:
                comm = 'search'
        # Check if the human prompt starts with "//rag "
        if last_line.startswith('//rag '):
            query = last_line[5:].strip()
            if query != None:
                comm = 'rag'
    return comm, query

def load_html():
    html = ''
    with open('index.html', 'r') as file:
        html = file.read()
    return html

def chat(prompt):
    modelId = 'ai21.j2-ultra'
    accept = 'application/json'
    contentType = 'application/json'
    body=json.dumps({'prompt': prompt, 'maxTokens': 250, 'stopSequences': ['Human:']})
    response = bedrock.invoke_model(body=body, modelId=modelId, accept=accept,contentType=contentType)
    response_body = json.loads(response.get('body').read())
    completions = response_body['completions']
    output = ''
    for part in completions:
        output += part['data']['text']
    return output

def search(query, limit=1):
    # get embedding
    embedding = get_embedding(query)
    # prepare for OpenSearch Serverless
    service = 'aoss'
    credentials = boto3.Session().get_credentials()
    awsauth = AWS4Auth(
        credentials.access_key, 
        credentials.secret_key, 
        region, 
        service, 
        session_token=credentials.token
    )
    # search
    index = 'demo-index'
    datatype = '_search'
    url = aoss_host + '/' + index + '/' + datatype
    headers = {'Content-Type': 'application/json'}
    document = {
        'size': limit,
        'query': {
            'knn': {
                'embedding': {
                    'vector': embedding,
                    'k': limit
                }
            }
        }
    }
    # response
    response = requests.get(url, auth=awsauth, json=document, headers=headers)
    response.raise_for_status()
    data = response.json()
    output = ''
    for item in data['hits']['hits']:
        output += item['_source']['content'] + '\n'
    return output.strip()

def get_embedding(text):
    modelId = 'amazon.titan-embed-text-v1'
    accept = 'application/json'
    contentType = 'application/json'
    input = {
            'inputText': text
        }
    body=json.dumps(input)
    response = bedrock.invoke_model(
        body=body, modelId=modelId, accept=accept,contentType=contentType)
    response_body = json.loads(response.get('body').read())
    embedding = response_body['embedding']
    return embedding


La lógica del código es:

- Para todas las solicitudes GET, cargamos el contenido HTML de index.html y se lo devolvemos al solicitante.
- Para todas las solicitudes POST, extraemos el cuerpo de la solicitud y, a continuación,
- Si la última línea de human comienza con , esto se trata como un comando de búsqueda, cualquier cosa después del comando se trata como el texto de la consulta. En este caso, convertimos el texto de la consulta en incrustación y, a continuación, buscamos en la base de conocimiento. Las 5 entradas de datos más cercanas en la base de conocimiento se devuelven al solicitante.//search//search
- Si la última línea de human comienza con , esto se trata como una solicitud de RAG. Todo lo que esté después del comando se trata como el texto de la consulta. En este caso, convertimos el texto de la consulta en incrustación, recuperamos las 5 entradas más relevantes de la base de conocimiento, aumentamos la solicitud con el contexto, invocamos el modelo base con la solicitud aumentada y devolvemos la respuesta al solicitante.//rag//rag
- De lo contrario, usamos el cuerpo de la solicitud como solicitud para invocar un modelo básico y, a continuación, devolvemos la respuesta del modelo básico al solicitante.
- Para todas las demás solicitudes, simplemente devolvemos un OK al solicitante.

**Paquete de implementación**

En la ventana del terminal, vaya a la carpeta Chatbot:

In [ ]:
cd ~/Chatbot

#Ejecute los siguientes comandos para volver a compilar el paquete de implementación:
rm my_deployment_package.zip
cd package
zip -r ../my_deployment_package.zip .
cd ..
zip my_deployment_package.zip index.html
zip my_deployment_package.zip chatbot_v1.py
zip my_deployment_package.zip chatbot_v2.py
zip my_deployment_package.zip chatbot_v3.py

#Copie el archivo zip en su bucket de S3.
S3_BUCKET=$(aws s3 ls | grep bedrock-workshop | cut -d' ' -f3-)
aws s3 cp my_deployment_package.zip s3://$S3_BUCKET

#Despliegue
#Ahora tenemos que configurar la función de Lambda para utilizar el nuevo paquete de implementación:
aws lambda update-function-code --function-name BedrockWorkshopChatbot --s3-bucket $S3_BUCKET --s3-key my_deployment_package.zip

#También necesitamos configurar la función Lambda para usar el controlador en chatbot_v3.py.
aws lambda update-function-configuration --function-name BedrockWorkshopChatbot --handler chatbot_v3.lambda_handler

#Acceda nuevamente a la interfaz de usuario web de su chatbot, pruebe con las siguientes indicaciones una por una para ver cómo funcionan las cosas.
#Comienza con un simple saludo:
